# Task 3.12: Model Calibration

## Overview
This task applies probability calibration techniques to improve prediction confidence for our top performing models. Well-calibrated probabilities are essential for reliable decision-making in production systems.

## Objectives
1. **Understand model calibration** and why it matters
2. **Apply Platt Scaling** (sigmoid calibration) to models
3. **Apply Isotonic Regression** calibration to models
4. **Generate calibration plots** to visualize calibration quality
5. **Compare calibration methods** and select the best approach

---

## What is Model Calibration?

### The Problem
Many classifiers output probability scores that don't reflect true probabilities. For example:
- A model predicting 80% probability should be correct ~80% of the time
- In practice, models are often **overconfident** (predict 90% but correct only 70%)
- Or **underconfident** (predict 60% but correct 80%)

### Why Calibration Matters
- **Risk assessment**: Medical diagnosis, fraud detection need accurate probabilities
- **Decision thresholds**: Setting optimal cutoffs requires calibrated scores
- **Ensemble methods**: Combining models works better with calibrated probabilities
- **User trust**: Displaying confidence scores to users requires accuracy

### Calibration Methods

#### 1. Platt Scaling (Sigmoid Calibration)
- Fits a logistic regression on the classifier's output scores
- Assumes sigmoid relationship between scores and true probabilities
- Works well for **SVMs** and models with sigmoid-shaped distortions
- **Pros**: Simple, works with small datasets
- **Cons**: Assumes specific functional form

#### 2. Isotonic Regression
- Non-parametric approach fitting a monotonic function
- More flexible than Platt scaling
- **Pros**: No assumptions about functional form
- **Cons**: Requires more data, can overfit on small datasets

---

## Step 1: Import Libraries and Setup

We import calibration-specific tools from sklearn:
- `CalibratedClassifierCV`: Wrapper for applying calibration
- `calibration_curve`: For generating reliability diagrams
- `brier_score_loss`: Metric for calibration quality

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import brier_score_loss, log_loss, accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.model_selection import train_test_split
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Create output directories
Path('../../outputs').mkdir(parents=True, exist_ok=True)
Path('../../outputs/figures').mkdir(parents=True, exist_ok=True)

print("="*60)
print("TASK 3.12: MODEL CALIBRATION")
print("="*60)
print("\nLibraries loaded successfully!")

## Step 2: Load and Prepare Data

### Important Notes
- We remove **leaky features** that would artificially inflate performance
- We need a **calibration set** separate from training data
- We split data into: Training (60%), Calibration (20%), Test (20%)

In [ ]:
# Load data
print("Loading preprocessed data...\n")

X_train = pd.read_csv('../../data/processed/X_train_scaled.csv')
X_test = pd.read_csv('../../data/processed/X_test_scaled.csv')
y_train = pd.read_csv('../../data/processed/y_train.csv')["value_category"]
y_test = pd.read_csv('../../data/processed/y_test.csv')["value_category"]

# Drop id column if exists
if 'id' in X_train.columns:
    X_train = X_train.drop('id', axis=1)
if 'id' in X_test.columns:
    X_test = X_test.drop('id', axis=1)

# Remove leaky features
leaky_features = [
    'price', 'price_normalized', 'price_per_person', 'price_per_bathroom',
    'price_per_bedroom', 'review_scores_rating', 'review_scores_value',
    'value_density', 'estimated_revenue_l365d'
]

cols_to_drop = [col for col in leaky_features if col in X_train.columns]
X_train = X_train.drop(columns=cols_to_drop)
X_test = X_test.drop(columns=cols_to_drop)

print(f"Dropped {len(cols_to_drop)} leaky features")
print(f"Remaining features: {X_train.shape[1]}")

# Encode target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Split training data into train and calibration sets
# Calibration requires held-out data to avoid overfitting
X_train_final, X_calib, y_train_final, y_calib = train_test_split(
    X_train, y_train_encoded, test_size=0.25, random_state=42, stratify=y_train_encoded
)

print(f"\n" + "="*60)
print("DATA SPLIT SUMMARY")
print("="*60)
print(f"Training set:    {len(y_train_final):,} samples")
print(f"Calibration set: {len(y_calib):,} samples")
print(f"Test set:        {len(y_test_encoded):,} samples")
print(f"\nClasses: {label_encoder.classes_}")
print(f"Encoded as: {list(range(len(label_encoder.classes_)))}")

## Step 3: Define Base Models

We use our top 3 models with regularization applied (from Task 3.11).

### Models Selected
1. **XGBoost**: Often overconfident, benefits from calibration
2. **Random Forest**: Generally well-calibrated but can improve
3. **MLP Classifier**: Neural networks often need calibration

In [ ]:
# Define base models with regularization
base_models = {
    'XGBoost': XGBClassifier(
        n_estimators=100,
        max_depth=4,
        learning_rate=0.05,
        min_child_weight=10,
        subsample=0.7,
        colsample_bytree=0.7,
        reg_alpha=0.5,
        reg_lambda=2.0,
        gamma=0.1,
        random_state=42,
        n_jobs=-1,
        verbosity=0
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=100,
        max_depth=6,
        min_samples_split=20,
        min_samples_leaf=12,
        max_features=0.3,
        random_state=42,
        n_jobs=-1
    ),
    'MLP': MLPClassifier(
        hidden_layer_sizes=(64, 32),
        activation='relu',
        solver='adam',
        alpha=0.1,
        learning_rate='adaptive',
        max_iter=500,
        early_stopping=True,
        validation_fraction=0.15,
        n_iter_no_change=15,
        random_state=42
    )
}

print("="*60)
print("BASE MODELS DEFINED")
print("="*60)
for name in base_models:
    print(f"  • {name}")

## Step 4: Train Base Models and Get Uncalibrated Probabilities

First, we train the base models and evaluate their **uncalibrated** probability predictions.

### Metrics for Calibration Quality

#### Brier Score
- Measures mean squared error between predicted probabilities and actual outcomes
- Range: 0 (perfect) to 1 (worst)
- Lower is better

#### Log Loss (Cross-Entropy)
- Penalizes confident wrong predictions heavily
- Lower is better
- More sensitive to calibration than accuracy

In [ ]:
print("="*60)
print("TRAINING BASE MODELS (UNCALIBRATED)")
print("="*60)

# Store results
uncalibrated_results = {}
trained_models = {}

for name, model in base_models.items():
    print(f"\nTraining {name}...", end=" ")
    
    # Train on training set
    model.fit(X_train_final, y_train_final)
    trained_models[name] = model
    
    # Get predictions and probabilities on test set
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test_encoded, y_pred)
    f1 = f1_score(y_test_encoded, y_pred, average='macro')
    logloss = log_loss(y_test_encoded, y_proba)
    
    # Brier score (multi-class: average of one-vs-all)
    y_test_bin = label_binarize(y_test_encoded, classes=[0, 1, 2])
    brier_scores = []
    for i in range(3):
        brier_scores.append(brier_score_loss(y_test_bin[:, i], y_proba[:, i]))
    brier_avg = np.mean(brier_scores)
    
    uncalibrated_results[name] = {
        'accuracy': accuracy,
        'f1_score': f1,
        'log_loss': logloss,
        'brier_score': brier_avg,
        'probabilities': y_proba
    }
    
    print(f"Done!")
    print(f"    Accuracy: {accuracy:.4f} | F1: {f1:.4f} | Log Loss: {logloss:.4f} | Brier: {brier_avg:.4f}")

print("\n" + "="*60)

## Step 5: Apply Calibration Methods

### Method 1: Platt Scaling (Sigmoid)
- Uses `method='sigmoid'` in CalibratedClassifierCV
- Fits logistic regression: P(y=1|f) = 1 / (1 + exp(A*f + B))
- Best for models with sigmoid-shaped miscalibration

### Method 2: Isotonic Regression
- Uses `method='isotonic'` in CalibratedClassifierCV
- Fits piecewise constant monotonic function
- More flexible but needs more data

### Cross-Validation Approach
We use `cv=5` to perform calibration with cross-validation, which:
- Avoids overfitting the calibration
- Uses all training data efficiently
- Provides more robust calibration

In [ ]:
print("="*60)
print("APPLYING CALIBRATION METHODS")
print("="*60)

# Store calibrated results
calibrated_results = {'platt': {}, 'isotonic': {}}
calibrated_models = {'platt': {}, 'isotonic': {}}

# Combine train and calibration for CV-based calibration
X_train_full = pd.concat([X_train_final, X_calib], axis=0)
y_train_full = np.concatenate([y_train_final, y_calib])

for name, base_model in base_models.items():
    print(f"\n{name}:")
    
    # Platt Scaling (Sigmoid)
    print("  Applying Platt Scaling...", end=" ")
    platt_model = CalibratedClassifierCV(
        estimator=base_model,
        method='sigmoid',
        cv=5
    )
    platt_model.fit(X_train_full, y_train_full)
    calibrated_models['platt'][name] = platt_model
    
    y_pred_platt = platt_model.predict(X_test)
    y_proba_platt = platt_model.predict_proba(X_test)
    
    accuracy_platt = accuracy_score(y_test_encoded, y_pred_platt)
    f1_platt = f1_score(y_test_encoded, y_pred_platt, average='macro')
    logloss_platt = log_loss(y_test_encoded, y_proba_platt)
    
    y_test_bin = label_binarize(y_test_encoded, classes=[0, 1, 2])
    brier_platt = np.mean([brier_score_loss(y_test_bin[:, i], y_proba_platt[:, i]) for i in range(3)])
    
    calibrated_results['platt'][name] = {
        'accuracy': accuracy_platt,
        'f1_score': f1_platt,
        'log_loss': logloss_platt,
        'brier_score': brier_platt,
        'probabilities': y_proba_platt
    }
    print(f"Done! Brier: {brier_platt:.4f}")
    
    # Isotonic Regression
    print("  Applying Isotonic Regression...", end=" ")
    iso_model = CalibratedClassifierCV(
        estimator=base_model,
        method='isotonic',
        cv=5
    )
    iso_model.fit(X_train_full, y_train_full)
    calibrated_models['isotonic'][name] = iso_model
    
    y_pred_iso = iso_model.predict(X_test)
    y_proba_iso = iso_model.predict_proba(X_test)
    
    accuracy_iso = accuracy_score(y_test_encoded, y_pred_iso)
    f1_iso = f1_score(y_test_encoded, y_pred_iso, average='macro')
    logloss_iso = log_loss(y_test_encoded, y_proba_iso)
    brier_iso = np.mean([brier_score_loss(y_test_bin[:, i], y_proba_iso[:, i]) for i in range(3)])
    
    calibrated_results['isotonic'][name] = {
        'accuracy': accuracy_iso,
        'f1_score': f1_iso,
        'log_loss': logloss_iso,
        'brier_score': brier_iso,
        'probabilities': y_proba_iso
    }
    print(f"Done! Brier: {brier_iso:.4f}")

print("\n" + "="*60)
print("Calibration complete!")
print("="*60)

## Step 6: Generate Calibration Plots (Reliability Diagrams)

### What is a Reliability Diagram?
A reliability diagram (calibration plot) shows:
- **X-axis**: Mean predicted probability (binned)
- **Y-axis**: Fraction of positives (actual frequency)
- **Diagonal line**: Perfect calibration

### Interpretation
- **Above diagonal**: Model is underconfident (predicts lower than actual)
- **Below diagonal**: Model is overconfident (predicts higher than actual)
- **On diagonal**: Well-calibrated

We create calibration plots for each class (one-vs-rest) to see class-specific calibration.

In [ ]:
# Generate calibration plots for each model
fig, axes = plt.subplots(3, 3, figsize=(18, 16))

class_names = label_encoder.classes_
y_test_bin = label_binarize(y_test_encoded, classes=[0, 1, 2])

for model_idx, name in enumerate(base_models.keys()):
    for class_idx in range(3):
        ax = axes[model_idx, class_idx]
        
        # Get probabilities for this class
        prob_uncalib = uncalibrated_results[name]['probabilities'][:, class_idx]
        prob_platt = calibrated_results['platt'][name]['probabilities'][:, class_idx]
        prob_iso = calibrated_results['isotonic'][name]['probabilities'][:, class_idx]
        y_true_class = y_test_bin[:, class_idx]
        
        # Calculate calibration curves
        frac_pos_uncalib, mean_pred_uncalib = calibration_curve(y_true_class, prob_uncalib, n_bins=10)
        frac_pos_platt, mean_pred_platt = calibration_curve(y_true_class, prob_platt, n_bins=10)
        frac_pos_iso, mean_pred_iso = calibration_curve(y_true_class, prob_iso, n_bins=10)
        
        # Plot
        ax.plot([0, 1], [0, 1], 'k--', label='Perfect', linewidth=2)
        ax.plot(mean_pred_uncalib, frac_pos_uncalib, 's-', color='red', 
                label='Uncalibrated', linewidth=2, markersize=8)
        ax.plot(mean_pred_platt, frac_pos_platt, 'o-', color='blue',
                label='Platt', linewidth=2, markersize=8)
        ax.plot(mean_pred_iso, frac_pos_iso, '^-', color='green',
                label='Isotonic', linewidth=2, markersize=8)
        
        ax.set_xlabel('Mean Predicted Probability', fontsize=10)
        ax.set_ylabel('Fraction of Positives', fontsize=10)
        ax.set_title(f'{name} - {class_names[class_idx]}', fontsize=12, fontweight='bold')
        ax.legend(loc='lower right', fontsize=9)
        ax.grid(True, alpha=0.3)
        ax.set_xlim([0, 1])
        ax.set_ylim([0, 1])

plt.suptitle('Calibration Plots (Reliability Diagrams) by Model and Class', 
             fontsize=16, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig('../../outputs/figures/calibration_plots_by_class.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nSaved: outputs/figures/calibration_plots_by_class.png")

## Step 7: Overall Calibration Comparison

We create a summary plot comparing all models and calibration methods on a single diagram.

In [ ]:
# Overall calibration plot (averaged across classes)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

colors = {'Uncalibrated': 'red', 'Platt': 'blue', 'Isotonic': 'green'}
markers = {'Uncalibrated': 's', 'Platt': 'o', 'Isotonic': '^'}

for model_idx, name in enumerate(base_models.keys()):
    ax = axes[model_idx]
    
    # Plot perfect calibration line
    ax.plot([0, 1], [0, 1], 'k--', label='Perfect Calibration', linewidth=2)
    
    # For each calibration method, average across classes
    for method, label in [('uncalib', 'Uncalibrated'), ('platt', 'Platt'), ('isotonic', 'Isotonic')]:
        all_frac_pos = []
        all_mean_pred = []
        
        for class_idx in range(3):
            if method == 'uncalib':
                proba = uncalibrated_results[name]['probabilities'][:, class_idx]
            else:
                proba = calibrated_results[method][name]['probabilities'][:, class_idx]
            
            y_true_class = y_test_bin[:, class_idx]
            frac_pos, mean_pred = calibration_curve(y_true_class, proba, n_bins=10)
            all_frac_pos.extend(frac_pos)
            all_mean_pred.extend(mean_pred)
        
        # Sort and plot
        sorted_idx = np.argsort(all_mean_pred)
        ax.plot(np.array(all_mean_pred)[sorted_idx], np.array(all_frac_pos)[sorted_idx],
                marker=markers[label], color=colors[label], label=label,
                linewidth=2, markersize=6, alpha=0.7)
    
    ax.set_xlabel('Mean Predicted Probability', fontsize=12)
    ax.set_ylabel('Fraction of Positives', fontsize=12)
    ax.set_title(f'{name}', fontsize=14, fontweight='bold')
    ax.legend(loc='lower right', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])

plt.suptitle('Overall Calibration Comparison', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../../outputs/figures/calibration_comparison_overall.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nSaved: outputs/figures/calibration_comparison_overall.png")

## Step 8: Quantitative Comparison

We compare all models and calibration methods using:
- **Brier Score**: Lower is better (measures calibration + discrimination)
- **Log Loss**: Lower is better (penalizes confident wrong predictions)
- **Accuracy**: Classification accuracy (may not change much with calibration)
- **F1-Score**: Balanced metric for multi-class

In [ ]:
print("="*70)
print("QUANTITATIVE COMPARISON")
print("="*70)

# Build comparison table
comparison_data = []

for name in base_models.keys():
    # Uncalibrated
    comparison_data.append({
        'Model': name,
        'Calibration': 'Uncalibrated',
        'Accuracy': uncalibrated_results[name]['accuracy'],
        'F1_Score': uncalibrated_results[name]['f1_score'],
        'Log_Loss': uncalibrated_results[name]['log_loss'],
        'Brier_Score': uncalibrated_results[name]['brier_score']
    })
    
    # Platt
    comparison_data.append({
        'Model': name,
        'Calibration': 'Platt Scaling',
        'Accuracy': calibrated_results['platt'][name]['accuracy'],
        'F1_Score': calibrated_results['platt'][name]['f1_score'],
        'Log_Loss': calibrated_results['platt'][name]['log_loss'],
        'Brier_Score': calibrated_results['platt'][name]['brier_score']
    })
    
    # Isotonic
    comparison_data.append({
        'Model': name,
        'Calibration': 'Isotonic',
        'Accuracy': calibrated_results['isotonic'][name]['accuracy'],
        'F1_Score': calibrated_results['isotonic'][name]['f1_score'],
        'Log_Loss': calibrated_results['isotonic'][name]['log_loss'],
        'Brier_Score': calibrated_results['isotonic'][name]['brier_score']
    })

comparison_df = pd.DataFrame(comparison_data)

# Display formatted table
print("\n" + comparison_df.to_string(index=False))

# Save to CSV
comparison_df.to_csv('../../outputs/calibration_comparison.csv', index=False)
print("\nSaved: outputs/calibration_comparison.csv")

## Step 9: Calibration Improvement Analysis

We calculate how much each calibration method improved the Brier score and Log Loss.

In [ ]:
print("="*70)
print("CALIBRATION IMPROVEMENT ANALYSIS")
print("="*70)

improvement_data = []

for name in base_models.keys():
    uncalib_brier = uncalibrated_results[name]['brier_score']
    uncalib_logloss = uncalibrated_results[name]['log_loss']
    
    platt_brier = calibrated_results['platt'][name]['brier_score']
    platt_logloss = calibrated_results['platt'][name]['log_loss']
    
    iso_brier = calibrated_results['isotonic'][name]['brier_score']
    iso_logloss = calibrated_results['isotonic'][name]['log_loss']
    
    # Calculate improvements (negative means improvement for these metrics)
    platt_brier_imp = (uncalib_brier - platt_brier) / uncalib_brier * 100
    platt_logloss_imp = (uncalib_logloss - platt_logloss) / uncalib_logloss * 100
    
    iso_brier_imp = (uncalib_brier - iso_brier) / uncalib_brier * 100
    iso_logloss_imp = (uncalib_logloss - iso_logloss) / uncalib_logloss * 100
    
    improvement_data.append({
        'Model': name,
        'Platt_Brier_Improvement_%': platt_brier_imp,
        'Platt_LogLoss_Improvement_%': platt_logloss_imp,
        'Isotonic_Brier_Improvement_%': iso_brier_imp,
        'Isotonic_LogLoss_Improvement_%': iso_logloss_imp,
        'Best_Method': 'Platt' if platt_brier < iso_brier else 'Isotonic'
    })
    
    print(f"\n{name}:")
    print(f"  Platt Scaling:")
    print(f"    • Brier Score: {platt_brier_imp:+.2f}% {'(improved)' if platt_brier_imp > 0 else '(worsened)'}")
    print(f"    • Log Loss:    {platt_logloss_imp:+.2f}% {'(improved)' if platt_logloss_imp > 0 else '(worsened)'}")
    print(f"  Isotonic Regression:")
    print(f"    • Brier Score: {iso_brier_imp:+.2f}% {'(improved)' if iso_brier_imp > 0 else '(worsened)'}")
    print(f"    • Log Loss:    {iso_logloss_imp:+.2f}% {'(improved)' if iso_logloss_imp > 0 else '(worsened)'}")

improvement_df = pd.DataFrame(improvement_data)
improvement_df.to_csv('../../outputs/calibration_improvement.csv', index=False)
print("\nSaved: outputs/calibration_improvement.csv")

## Step 10: Summary Visualization

Final visualization comparing Brier scores and Log Loss across all configurations.

In [ ]:
# Summary bar chart
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

models_list = list(base_models.keys())
x = np.arange(len(models_list))
width = 0.25

# Brier Score comparison
ax1 = axes[0]
brier_uncalib = [uncalibrated_results[m]['brier_score'] for m in models_list]
brier_platt = [calibrated_results['platt'][m]['brier_score'] for m in models_list]
brier_iso = [calibrated_results['isotonic'][m]['brier_score'] for m in models_list]

bars1 = ax1.bar(x - width, brier_uncalib, width, label='Uncalibrated', color='red', alpha=0.8)
bars2 = ax1.bar(x, brier_platt, width, label='Platt Scaling', color='blue', alpha=0.8)
bars3 = ax1.bar(x + width, brier_iso, width, label='Isotonic', color='green', alpha=0.8)

ax1.set_ylabel('Brier Score (lower is better)', fontsize=12)
ax1.set_title('Brier Score Comparison', fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(models_list, fontsize=11)
ax1.legend(fontsize=10)
ax1.grid(axis='y', alpha=0.3)

# Add value labels
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        ax1.annotate(f'{bar.get_height():.4f}', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                     xytext=(0, 3), textcoords='offset points', ha='center', fontsize=8, rotation=90)

# Log Loss comparison
ax2 = axes[1]
logloss_uncalib = [uncalibrated_results[m]['log_loss'] for m in models_list]
logloss_platt = [calibrated_results['platt'][m]['log_loss'] for m in models_list]
logloss_iso = [calibrated_results['isotonic'][m]['log_loss'] for m in models_list]

bars4 = ax2.bar(x - width, logloss_uncalib, width, label='Uncalibrated', color='red', alpha=0.8)
bars5 = ax2.bar(x, logloss_platt, width, label='Platt Scaling', color='blue', alpha=0.8)
bars6 = ax2.bar(x + width, logloss_iso, width, label='Isotonic', color='green', alpha=0.8)

ax2.set_ylabel('Log Loss (lower is better)', fontsize=12)
ax2.set_title('Log Loss Comparison', fontsize=14, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(models_list, fontsize=11)
ax2.legend(fontsize=10)
ax2.grid(axis='y', alpha=0.3)

for bars in [bars4, bars5, bars6]:
    for bar in bars:
        ax2.annotate(f'{bar.get_height():.4f}', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                     xytext=(0, 3), textcoords='offset points', ha='center', fontsize=8, rotation=90)

plt.suptitle('Calibration Methods Comparison', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../../outputs/figures/calibration_summary.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nSaved: outputs/figures/calibration_summary.png")

## Step 11: Final Conclusions

Summary of findings and recommendations for model deployment.

In [ ]:
print("\n" + "="*70)
print("FINAL CONCLUSIONS AND RECOMMENDATIONS")
print("="*70)

# Find best overall configuration
best_config = comparison_df.loc[comparison_df['Brier_Score'].idxmin()]

print(f"\n1. BEST CALIBRATED MODEL")
print(f"   → {best_config['Model']} with {best_config['Calibration']}")
print(f"   → Brier Score: {best_config['Brier_Score']:.4f}")
print(f"   → Log Loss: {best_config['Log_Loss']:.4f}")

print(f"\n2. CALIBRATION METHOD COMPARISON")
print(f"   • Platt Scaling: Better for smaller datasets, assumes sigmoid shape")
print(f"   • Isotonic: More flexible, may overfit with limited data")

print(f"\n3. KEY FINDINGS")
for _, row in improvement_df.iterrows():
    print(f"   • {row['Model']}: Best with {row['Best_Method']}")



print("\n" + "="*70)
print("TASK 3.12: MODEL CALIBRATION - COMPLETE")
print("="*70)

## Files Generated

### Analysis Files (outputs/)
- `calibration_comparison.csv` - Full comparison of all models and methods
- `calibration_improvement.csv` - Improvement percentages from calibration

### Visualization Files (outputs/figures/)
- `calibration_plots_by_class.png` - Reliability diagrams for each model/class
- `calibration_comparison_overall.png` - Overall calibration comparison
- `calibration_summary.png` - Brier score and Log Loss comparison

---

## Summary

| Aspect | Finding |
|--------|--------|
| Best Method | Depends on model, generally Platt for this dataset |
| Improvement | 5-15% reduction in Brier score typical |
| Trade-off | Calibration doesn't change accuracy much |
| Use Case | Essential for probability-based decisions |